In [1]:
import os
import numpy  as np
import pandas as pd

from antea  .io.mc_io    import load_configuration

### Checking size and configuration for every file:

In [2]:
params = ['thr_charge', 'sns_only', 'tof_pe_number']

files = np.array([1, 2, 3, 4])

conf_info = np.array([np.array([2, 'false', 10]), 
                      np.array([2,  'true', 10]), 
                      np.array([2,  'true', 20]), 
                      np.array([4,  'true', 10])])

params_df = pd.DataFrame(data   =conf_info, 
                         index  =files, 
                         columns=params)
print('Configuration for each file:')
params_df

Configuration for each file:


,thr_charge,sns_only,tof_pe_number
1,2,false,10
2,2,true,10
3,2,true,20
4,4,true,10


In [35]:
path  = '/data5/users/carmenromo/testing_nexus/data_ring/'
file1 = path+'full_ring_iradius165mm_z140mm_depth3cm_pitch7mm_test1.100.pet.h5'
file2 = path+'full_ring_iradius165mm_z140mm_depth3cm_pitch7mm_test3.000.pet.h5'
file3 = path+'full_ring_iradius165mm_z140mm_depth3cm_pitch7mm_test4.000.pet.h5'
file4 = path+'full_ring_iradius165mm_z140mm_depth3cm_pitch7mm_test5.000.pet.h5'
file5 = path+'full_ring_iradius165mm_z140mm_depth3cm_pitch7mm_test6.000.pet.h5'

total_size1 = os.path.getsize(file1)/1e6
total_size2 = os.path.getsize(file2)/1e6
total_size3 = os.path.getsize(file3)/1e6
total_size4 = os.path.getsize(file4)/1e6
total_size5 = os.path.getsize(file5)/1e6

h5conf1     = load_configuration(file1)
h5conf2     = load_configuration(file2)
h5conf3     = load_configuration(file3)
h5conf4     = load_configuration(file4)
h5conf5     = load_configuration(file5)

saved_evts1 = int(h5conf1[h5conf1.param_key=='saved_events'].param_value.values[0])
saved_evts2 = int(h5conf2[h5conf2.param_key=='saved_events'].param_value.values[0])
saved_evts3 = int(h5conf3[h5conf3.param_key=='saved_events'].param_value.values[0])
saved_evts4 = int(h5conf4[h5conf4.param_key=='saved_events'].param_value.values[0])
saved_evts5 = int(h5conf5[h5conf5.param_key=='saved_events'].param_value.values[0])

print('NEXUS initial file:')
print(f"{round(total_size1, 2)} Mb (15000 events simulated, {saved_evts1} saved)")
print('')
print('NEXUS file2:')
print(f"{round(total_size2, 2)} Mb (15000 events simulated, {saved_evts2} saved)")
print('')
print('NEXUS file3')
print(f"{round(total_size3, 2)} Mb (15000 events simulated, {saved_evts3} saved)")
print('')
print('NEXUS file4')
print(f"{round(total_size4, 2)} Mb (15000 events simulated, {saved_evts4} saved)")
print('')
print('NEXUS file5:')
print(f"{round(total_size5, 2)} Mb (15000 events simulated, {saved_evts5} saved)")
print('')

ratio1 = 1 - total_size2/total_size1
ratio2 = 1 - total_size3/total_size1
ratio3 = 1 - total_size4/total_size1
ratio4 = 1 - total_size5/total_size1

print(f"File2 is {round(ratio1, 2)}% lighter")
print(f"File3 is {round(ratio2, 2)}% lighter")
print(f"File4 is {round(ratio3, 2)}% lighter")
print(f"File5 is {round(ratio4, 2)}% lighter")

NEXUS initial file:
280.91 Mb (15000 events simulated, 7007 saved)

NEXUS file2:
213.15 Mb (15000 events simulated, 7007 saved)

NEXUS file3
84.17 Mb (15000 events simulated, 7007 saved)

NEXUS file4
97.28 Mb (15000 events simulated, 7007 saved)

NEXUS file5:
68.96 Mb (15000 events simulated, 7007 saved)

File2 is 0.24% lighter
File3 is 0.7% lighter
File4 is 0.65% lighter
File5 is 0.75% lighter


### Checking that the tables are equal depending on the case:

In [4]:
from antea.io.mc_io import load_mcsns_response
from antea.io.mc_io import load_mcTOFsns_response
from antea.io.mc_io import load_mcparticles
from antea.io.mc_io import load_mchits

#### No true info:

In [36]:
files = np.array([file2, file3, file4, file5])

for i,(j,df_row) in zip(files, params_df.iterrows()):
    sns_response     = load_mcsns_response   (i)
    sns_tof_response = load_mcTOFsns_response(i)
    particles        = load_mcparticles      (i)
    hits             = load_mchits           (i)
    
    if df_row.sns_only == 'true':
        assert particles.empty
        assert hits     .empty

#### Charges above thr

In [39]:
sns_response1 = load_mcsns_response(file1)
sns_response3 = load_mcsns_response(file3)
sns_response5 = load_mcsns_response(file5)

In [40]:
df_above_thr3 = sns_response3[sns_response3.charge > 4].reset_index()
df_above_thr3 = df_above_thr3.drop('index', axis=1)

assert len(df_above_thr3) == len(sns_response5)
print(df_above_thr3.equals(sns_response5))

True


In [41]:
df_above_thr1 = sns_response1[sns_response1.charge > 2].reset_index()
df_above_thr1 = df_above_thr1.drop('index', axis=1)

assert len(df_above_thr1) == len(sns_response2)
assert len(df_above_thr1) == len(sns_response3)
assert len(df_above_thr1) == len(sns_response4)
print(df_above_thr1.equals(sns_response2))
print(df_above_thr1.equals(sns_response3))
print(df_above_thr1.equals(sns_response4))

False
False
False


#### bin sizes:

In [23]:
bin_size0     = h5conf1[h5conf1.param_key==    'bin_size'].param_value.values[0]
tof_bin_size0 = h5conf1[h5conf1.param_key=='tof_bin_size'].param_value.values[0]
for i,(j,df_row) in zip(files, params_df.iterrows()):
    h5config     = load_configuration(i)
    bin_size     = h5config[h5config.param_key==    'bin_size'].param_value.values[0]
    tof_bin_size = h5config[h5config.param_key=='tof_bin_size'].param_value.values[0]
    assert bin_size     == bin_size0
    assert tof_bin_size == tof_bin_size0

#### Assert all the config parameters except the changed ones are the same:

In [112]:
for i,j,k,l,m,n,o,p,q,r in zip(h5conf1.param_key, h5conf1.param_value, h5conf2.param_key, h5conf2.param_value,
                               h5conf3.param_key, h5conf3.param_value, h5conf4.param_key, h5conf4.param_value,
                               h5conf5.param_key, h5conf5.param_value):
    if i=='/nexus/RegisterMacro' or i=='/nexus/persistency/outputFile':
        continue
    arr = np.array([j,l,n,p,r])
    #print(i,k)
    if not np.all(arr == arr[0]):
        print(i)
        print(arr)
        
print(len(h5conf1), len(h5conf2))

/nexus/persistency/start_id
['1500000' '0' '0' '0' '0']
/nexus/persistency/hdf5
['true' '2' '2' '2' '4']
40 42


### Tof

In [139]:
sns_response_tof1 = load_mcTOFsns_response(file1)
sns_response_tof2 = load_mcTOFsns_response(file2)

particles2 = load_mcparticles(file2)
events     = particles2.event_id.unique()
for evt in range(1):
    waveforms2     = sns_response2[sns_response2.event_id == evt]
    sipms2         = waveforms2.sensor_id.values.astype(np.int32)
    
    tof_waveforms2 = sns_response_tof2[sns_response_tof2.event_id == evt].reset_index()
    tof_waveforms2 = tof_waveforms2.drop('index', axis=1)
    
    tof_waveforms1 = sns_response_tof1[sns_response_tof1.event_id == evt+1500000]
    sipms_tof1     = tof_waveforms1.sensor_id.values
    for s_id in sipms2:
        see  = tof_waveforms1[tof_waveforms1.sensor_id==-s_id].iloc[:10].drop('event_id', axis=1).reset_index().drop('index', axis=1)
        see2 = tof_waveforms2[tof_waveforms2.sensor_id==-s_id]          .drop('event_id', axis=1).reset_index().drop('index', axis=1)
        if not see.equals(see2):
            print(see)
            print(see2)
            print('')